In [7]:
# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error


In [19]:
# REQUIRED IMPORTS
import pandas as pd

# LOAD DATASETS
train = pd.read_csv("train.csv", parse_dates=["Date"], low_memory=False)
test = pd.read_csv("test.csv", parse_dates=["Date"], low_memory=False)
store = pd.read_csv("store.csv")

# MERGE STORE DETAILS
train = train.merge(store, on="Store", how="left")
test = test.merge(store, on="Store", how="left")

# FEATURE ENGINEERING (same for both train & test)
for df in [train, test]:
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["WeekOfYear"] = df["Date"].dt.isocalendar().week.astype(int)
    df["IsWeekend"] = (df["DayOfWeek"] > 5).astype(int)
    df["CompetitionDistance"] = df["CompetitionDistance"].fillna(200000)

# FILTER TRAIN MODEL DATA
train_model = train[(train["Open"] == 1) & (train["Sales"] > 0)]

# ENSURE CUSTOMERS COLUMN
test["Customers"] = 0

print("Train shape:", train.shape)
print("Train_model shape:", train_model.shape)
print("Test shape:", test.shape)


Train shape: (1017209, 23)
Train_model shape: (844338, 23)
Test shape: (41088, 23)


In [9]:


features = ["Store", "DayOfWeek", "Promo", "Month", "Year", "Customers"]

X = train_model[features].fillna(0)
y = train_model["Sales"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train_dl, X_val_dl, y_train_dl, y_val_dl = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Deep learning-like model
model_dl = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    solver="adam",
    max_iter=20,
    random_state=42,
    verbose=True
)

# Train the model
model_dl.fit(X_train_dl, y_train_dl)

# Evaluate
preds = model_dl.predict(X_val_dl)
mae_dl = mean_absolute_error(y_val_dl, preds)
print("Deep Learning Model MAE:", mae_dl)


Iteration 1, loss = 4263558.56935290
Iteration 2, loss = 1212051.10595917
Iteration 3, loss = 1134495.13978930
Iteration 4, loss = 1120614.62425853
Iteration 5, loss = 1114036.23586631
Iteration 6, loss = 1108994.74126497
Iteration 7, loss = 1104249.83444527
Iteration 8, loss = 1099597.67547501
Iteration 9, loss = 1095445.10275678
Iteration 10, loss = 1092184.76631604
Iteration 11, loss = 1089014.41589634
Iteration 12, loss = 1086393.95299579
Iteration 13, loss = 1084354.96381154
Iteration 14, loss = 1082548.17942187
Iteration 15, loss = 1080656.24383583
Iteration 16, loss = 1078965.36612027
Iteration 17, loss = 1077509.36678108
Iteration 18, loss = 1076048.66944449
Iteration 19, loss = 1074643.03692922
Iteration 20, loss = 1073211.59537079


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Deep Learning Model MAE: 1063.9372776558105


STEP 2 — Generate Predictions Using the Deep Learning Model

In [21]:
test_scaled = scaler.transform(test[features].fillna(0))
test_dl_preds = model_dl.predict(test_scaled)

submission_dl = pd.DataFrame({
    "Id": test["Id"],
    "Sales": test_dl_preds
})

submission_dl.to_csv("dl_predictions.csv", index=False)
submission_dl.head()


,Id,Sales
0,1,919.338856
1,2,918.061014
2,3,915.505328
3,4,914.866407
4,5,914.227485
